In [90]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table = soup.find(soup.find("table",{"class":"wikitable sortable jquery-tablesorter"}))
table_body = table.find('tbody')
rows = table_body.find_all('tr')

PostalCode = []
Borough = []
Neighborhood = []

for row in rows:
    column = [] 
    cols = row.find_all('td')
    for col in cols:
        column.append(col.get_text(strip=True))

    if len(column) != 0:
        if column[1] != "Not assigned":
            PostalCode.append(column[0])
            Borough.append(column[1])
            Neighborhood.append(column[2])

data = pd.DataFrame({"PostalCode": PostalCode, "Borough": Borough, "Neighborhood": Neighborhood}) 

data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [91]:
print(data.shape)

(103, 3)


Couldn't get geocoder to install, so using csv

In [92]:
import geocoder # import geocoder

postal_code = "M5G"

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'

In [93]:
geo = pd.read_csv("Geospatial_Coordinates.csv")

#Sort by postal code
data = data.sort_values("PostalCode", ascending = True)

data["Latitude"] = geo["Latitude"]
data["Longitude"] = geo["Longitude"]

print(data.loc[data["PostalCode"] == "M5G"])

   PostalCode           Borough        Neighborhood   Latitude  Longitude
24        M5G  Downtown Toronto  Central Bay Street  43.782736 -79.442259


In [97]:
import folium
from folium import plugins

map_clusters = folium.Map(location=[data["Latitude"].mean(), data["Longitude"].mean()], zoom_start=11)

neighbourhoods = plugins.MarkerCluster().add_to(map_clusters)
                                    
for lat, long in zip(data.Latitude, data.Longitude):
    folium.Marker(
            location=[lat, long],
            icon=None,
        ).add_to(neighbourhoods)

map_clusters